In [1]:
import matplotlib
import strax
st = strax.Context(register_all=strax.xenon.plugins)

import graphviz

plugins = st._get_plugins(('event_info',), run_id='0')

In [3]:
kind_colors = dict(
    events='white',
    peaks='palegreen',
    records='orange',
    raw_records='orangered')

In [4]:
import pandas as pd
def df_to_markdown(df):
    fmt = ['---' for i in range(len(df.columns))]
    df_fmt = pd.DataFrame([fmt], columns=df.columns)
    df_formatted = pd.concat([df_fmt, df])
    return df_formatted.to_csv(sep="|", index=False)

In [5]:
# Too lazy to write proper graph sorter
from collections import defaultdict
plugins_by_deps = defaultdict(list)

for pn, p in st._plugin_class_registry.items():
    plugins = st._get_plugins((pn,), run_id='0')
    plugins_by_deps[len(plugins)].append(pn)

In [6]:
import os
try:
    os.makedirs('graphs')
except FileExistsError:
    pass

In [7]:
#![{data_type}](./graphs/{data_type}.svg)

In [8]:
print("""

sdf 
sdf 
sdf 
""")



sdf 
sdf 
sdf 



In [9]:
def add_spaces(x):
    y = ''
    if isinstance(x, str):
        x = x.split('\n')
    for q in x:
        y += '    ' + q
    return y

In [13]:
pd.set_option('display.max_colwidth', -1)

In [24]:
import numpy as np

In [45]:
out = """
Strax XENON datastrure
========================

This page is an autogenerated reference for all the XENON plugins in strax. 

Colors indicate data kinds. To load tables with different data kinds,
you currently need more than one `get_df` (or `get_array`) commands.

"""

template = """
{data_type}
--------------------------------------------------------

Description
~~~~~~~~~~~~~~~~~~~~~~

Provided by plugin: {p.__class__.__name__}

Data kind: {p.data_kind}

{docstring}


Dependencies
~~~~~~~~~~~~~~~~~~~~~~
.. raw:: html

{svg}


Columns provided
~~~~~~~~~~~~~~~~~~~~~~
.. raw:: html

{columns}


Configuration options
~~~~~~~~~~~~~~~~~~~~~~~

These are all options that affect this data type. 
This also includes options taken by dependencies of this datatype,
because changing any of those options affect this data indirectly.

.. raw:: html

{config_options}


"""

for n_deps in list(reversed(sorted(list(plugins_by_deps.keys())))):
    for data_type in plugins_by_deps[n_deps]:

        plugins = st._get_plugins((data_type,), run_id='0')

        # Create dependency graph
        g = graphviz.Digraph(format='svg')
        for d, p in plugins.items():
            g.node(d, 
                   style='filled',
                   href='#' + d.replace('_', '-'),
                   fillcolor=matplotlib.colors.to_hex(
                       kind_colors[p.data_kind],
                   ))
            for dep in p.depends_on:
                g.edge(d, dep)
        fn = 'graphs/' + data_type
        g.render(fn)
        with open(fn + '.svg', mode='r') as f:
            svg = add_spaces(f.readlines()[5:])
            
        config_df = st.show_config(d).sort_values(by='option')
        
        # Shorten long default values
        config_df['default'] = [
            x[:10] + '...' + x[-10:]
            if isinstance(x, str) and len(x) > 30 else x
            for x in config_df['default'].values]

        p = plugins[data_type]
        doc = p.__doc__

        out += template.format(
            p=p,
            svg=svg,
            data_type = data_type,
            columns = add_spaces(
                st.data_info(data_type).to_html(index=False)
            ),
            docstring = p.__doc__ if p.__doc__ else '(no plugin description)',
            config_options=add_spaces(
                config_df.to_html(index=False))
        )

with open('datastructure.rst', mode='w') as f:
    f.write(out)

!rm -r graphs